In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [128]:
# parser v1
# parse records to list of dicts
# values mostly are strings
# easy to iterate through records
records = []
with open('Webscope_R6B/ydata-fp-td-clicks-v2_0.20111002') as f:
    for line in f.readlines():
        components = line.strip().split(' ')
        record = {}
        record['timestamp'] = components[0]
        record['displayed'] = components[1]
        record['clicked'] = int(components[2])
        # not using user feature
        record['user_features'] = []
        for i in range(4, len(components)):
            if components[i].startswith('|'):
                n = i
                break
            #record['user_features'].append(int(components[i]))
        record['pool'] = components[n:]    
        records.append(record)

In [152]:
len(records)

1607525

In [89]:
# parser that watches article lifespans
lives = {}
with open('Webscope_R6B/ydata-fp-td-clicks-v2_0.20111002') as f:
    for line in f.readlines():
        components = line.strip().split(' ')
        for component in components:
            if component.startswith('|id-'):
                timestamp = int(components[0])
                if component in lives.keys():
                    lives[component][1] = timestamp
                else:
                    lives[component] = [timestamp, timestamp]

In [22]:
lives.keys()

dict_keys(['|id-552077', '|id-555224', '|id-555528', '|id-559744', '|id-559855', '|id-560290', '|id-560518', '|id-560620', '|id-563115', '|id-563582', '|id-563643', '|id-563787', '|id-563846', '|id-563938', '|id-564335', '|id-564418', '|id-564604', '|id-565364', '|id-565479', '|id-565515', '|id-565533', '|id-565561', '|id-565589', '|id-565648', '|id-565747', '|id-565822', '|id-565930', '|id-566013', '|id-566022', '|id-566092', '|id-560805', '|id-564371', '|id-562265', '|id-565980', '|id-566431', '|id-566439', '|id-559833', '|id-566541', '|id-562374', '|id-566587', '|id-566478', '|id-566573', '|id-566602', '|id-562637', '|id-566631', '|id-566689', '|id-566726', '|id-566825', '|id-566838'])

In [125]:
for v in lives.values():
    print(v)

[1317513291, 1317515399]
[1317513291, 1317588299]
[1317513291, 1317587099]
[1317513291, 1317515699]
[1317513291, 1317587099]
[1317513291, 1317587099]
[1317513291, 1317578399]
[1317513291, 1317521099]
[1317513291, 1317581999]
[1317513291, 1317583799]
[1317513291, 1317599694]
[1317513291, 1317515699]
[1317513291, 1317599694]
[1317513291, 1317519299]
[1317513291, 1317571199]
[1317513291, 1317599694]
[1317513291, 1317520799]
[1317513291, 1317535499]
[1317513291, 1317583499]
[1317513291, 1317587399]
[1317513291, 1317583499]
[1317513291, 1317549899]
[1317513291, 1317577499]
[1317513291, 1317586799]
[1317513291, 1317570299]
[1317513291, 1317599694]
[1317517500, 1317582299]
[1317525000, 1317585899]
[1317527100, 1317594299]
[1317529500, 1317564299]
[1317565500, 1317599694]
[1317568200, 1317599694]
[1317569100, 1317599694]
[1317570600, 1317574799]
[1317572400, 1317599694]
[1317573300, 1317599694]
[1317577200, 1317599694]
[1317577800, 1317599694]
[1317578100, 1317599694]
[1317579300, 1317599694]


In [155]:
timestamp = 1317513600
life_rank = sorted(filter(lambda x: x[1][0]<timestamp, lives.items()), key= lambda x: x[1][1], reverse= True)
life_rank

[('|id-563643', [1317513291, 1317599694]),
 ('|id-563846', [1317513291, 1317599694]),
 ('|id-564418', [1317513291, 1317599694]),
 ('|id-565822', [1317513291, 1317599694]),
 ('|id-555224', [1317513291, 1317588299]),
 ('|id-565515', [1317513291, 1317587399]),
 ('|id-555528', [1317513291, 1317587099]),
 ('|id-559855', [1317513291, 1317587099]),
 ('|id-560290', [1317513291, 1317587099]),
 ('|id-565648', [1317513291, 1317586799]),
 ('|id-563582', [1317513291, 1317583799]),
 ('|id-565479', [1317513291, 1317583499]),
 ('|id-565533', [1317513291, 1317583499]),
 ('|id-563115', [1317513291, 1317581999]),
 ('|id-560518', [1317513291, 1317578399]),
 ('|id-565589', [1317513291, 1317577499]),
 ('|id-564335', [1317513291, 1317571199]),
 ('|id-565747', [1317513291, 1317570299]),
 ('|id-565561', [1317513291, 1317549899]),
 ('|id-565364', [1317513291, 1317535499]),
 ('|id-560620', [1317513291, 1317521099]),
 ('|id-564604', [1317513291, 1317520799]),
 ('|id-563938', [1317513291, 1317519299]),
 ('|id-5597

In [168]:
life_rank[:int(len(life_rank)*.3)]

[('|id-563643', [1317513291, 1317599694]),
 ('|id-563846', [1317513291, 1317599694]),
 ('|id-564418', [1317513291, 1317599694]),
 ('|id-565822', [1317513291, 1317599694]),
 ('|id-555224', [1317513291, 1317588299]),
 ('|id-565515', [1317513291, 1317587399]),
 ('|id-555528', [1317513291, 1317587099])]

In [194]:
# offline simulation for AG_L
def AG_L(n=1000000, m=25, life_threshold=.3):
    # records should be long enough
    '''
    n: number of turns to play
    m: number of turns to initialize
    '''
    global records
    global lives # dict of articles to their lifespans
    rewards = 0
    i = 0 # marker in records
    X = {} # dict of expectations article_id : array([sum, n_observations])
    t = 1 # number of turn taken
    
    # initialization
    while t<=m:
        displayed = records[i]['displayed']
        clicked = records[i]['clicked']
        if displayed not in X.keys():
            X[displayed] = np.array([clicked, 1])
            #print('initialized',displayed,':',X[displayed],'t/i:',t,i)
            t += 1
        i += 1
    
    n_exploit = 0
    n_explore = 0
    while t<=n:
        best = max(X.items(), key= lambda x:x[1][0]/x[1][1])
        X_max = best[1][0]/best[1][1]
        #print('Turn', t, 'X_max:', X_max)
        if np.random.binomial(1, X_max): # exploit
            n_exploit += 1
            while i < len(records): # look for target arm
                displayed = records[i]['displayed']
                clicked = records[i]['clicked']
                if displayed == best[0]:
                    rewards += clicked
                    #print('exploited', displayed, ':', clicked, 'i:', i)
                    break
                i += 1
                
        else: # explore
            n_explore += 1
            while i < len(records):
                displayed = records[i]['displayed']
                clicked = records[i]['clicked']
                time = int(records[i]['timestamp'])
                born = filter(lambda x: x[1][0]<time, lives.items()) # fileter out unborn articles from lives
                life_rank = sorted(born, key= lambda x: x[1][1], reverse= True) # sort born articles by their expiring date
                young = life_rank[ : int(len(life_rank)*life_threshold)] # articles among top $life_threshold$% on the life ranking
                if '|'+displayed in list(zip(*young))[0]:
                    rewards += clicked
                    #print('explored', displayed, ':', clicked, 'i:', i)
                    break
                i += 1

        X[displayed] = X.get(displayed, np.array([0, 0])) + np.array([clicked, 1])
        t += 1
        i += 1
        
    print('# exploit:', n_exploit)
    print('# explore:', n_explore)
    print('# records:', i)
    print('max expectation:', X_max)
    
    return rewards

In [195]:
AG_L(n=1000)

# exploit: 105
# explore: 870
# records: 6070
max expectation: 0.0752212389380531


48

In [180]:
from collections import Counter

In [181]:
c=Counter()
i=0
while i<1000:
    c[records[i]['clicked']] += 1
    i += 1

In [182]:
c

Counter({0: 965, 1: 35})